   Создание кластеров по баллам ЕГЭ и по баллам за 1 семестр: ищем 20 ближайших соседей там и там. Далее, проводим сравнение кластеров, то есть для каждого объекта смотрим как изменилось количество его ближайших соседей.
Подготовка: выводим информацию по каждой специальности 

In [30]:
#Функция для удаления дубликатов
def unique(lst):
    seen = set()
    result = []
    for x in lst:
        if x in seen:
            continue
        seen.add(x)
        result.append(x)
    return result


#Функция перемножения множеств
def inters(a):
    s=set(a[0])
    for lst in a:
        s=s.intersection(set(lst))
    return(s)   

In [31]:
#Ищем id студентов выбранной специальности
#вывести список групп
def spec():
    import sqlite3
    #Создание списка из всех направлений...
 
    conn = sqlite3.connect("stud_spisok_ege.db")
    cursor = conn.cursor()

    #выбираем прошедшие данные чтобы их вывести...
    sql = "SELECT grp_spec_grp_spec_name FROM stud_spisok "
    cursor.execute(sql)
    #print(cursor.fetchall())
    specials=cursor.fetchall()

    conn.commit()
    conn.close()

    #Делаем список из специальностей
    specs=[]
    for i in range(len(unique(specials))):
        specs.append(unique(specials)[i][0])
    return(specs)

In [32]:
#Ищем id студентов выбранной специальности
def kods(specs):
    import sqlite3
 
    conn = sqlite3.connect("stud_spisok_ege.db")
    cursor = conn.cursor()

    #выбираем прошедшие данные чтобы их вывести...
    sql = "SELECT stud_kod FROM stud_spisok WHERE grp_spec_grp_spec_name = ?"
    cursor.execute(sql,[(specs)])
    #print(cursor.fetchall())
    students_kod=cursor.fetchall()

    conn.commit()
    conn.close()

    kod=[]
    for i in range(len(students_kod)):
        kod.append(students_kod[i][0])
    return(kod)

In [34]:
kod=kods(spec()[0])
kod

[50095,
 50104,
 50137,
 50139,
 50152,
 50172,
 50193,
 50205,
 50209,
 50220,
 50222,
 50226,
 50230,
 50232,
 50239,
 50247,
 50248,
 50250,
 50280,
 50291,
 50308,
 50315,
 50336,
 50343,
 50350,
 50379,
 50481,
 50509,
 50513,
 50553,
 50557,
 50559,
 50563,
 50576,
 50610,
 50619,
 50627,
 50636,
 50655,
 50659,
 50662,
 50676,
 50689,
 50711,
 50712,
 50720,
 50724,
 50726,
 50728,
 50744,
 50748,
 50760,
 50763,
 50768,
 50787,
 50820,
 50828,
 50838,
 50855,
 50862,
 50867,
 50888,
 50890,
 50904,
 50920,
 50925,
 50926,
 50928,
 50934,
 50939,
 50940,
 51012,
 51049,
 51050,
 51073,
 51075,
 51076,
 51092,
 51142,
 51155,
 51161,
 51171,
 51172,
 51176,
 51180,
 51213,
 51223,
 51224,
 51233,
 51237,
 51242,
 51255,
 51262,
 51281,
 51299,
 51306,
 51350,
 51384,
 51387,
 51390,
 51402,
 51419,
 51454,
 51472,
 51494,
 51495,
 51510,
 51578,
 51586,
 51609,
 51625,
 51627,
 51639,
 51641,
 51642,
 51648,
 51705,
 51716,
 51726,
 51729,
 51731,
 51733,
 51782,
 51783,
 51789,


In [26]:
#Вывод баллов ЕГЭ для выбранного студента:
def identyEge(IDENTY, Ege_predmet):
    import sqlite3
 
    conn = sqlite3.connect("stud_spisok_ege.db")
    cursor = conn.cursor()

    #выбираем прошедшие данные чтобы их вывести...
    sql = "SELECT ball FROM raiting_ege WHERE stud_kod = ? AND predmet_name=?"
    cursor.execute(sql,[(IDENTY),(Ege_predmet)])
    #print(cursor.fetchall())
    student_ege=cursor.fetchall()
    conn.commit()
    conn.close()
    if student_ege==[]:return(0)
    else: return(student_ege[0][0])

In [50]:
identyEge(kod[0], 'Русский язык')

76

In [51]:
#ВЫбираем 3 балла ЕГэ по каждому студенту:
ege_spis=[]
for i in range(len(kod)):
    ege_spis.append([kod[i],identyEge(kod[i], 'Русский язык'),identyEge(kod[i], 'Математика'),identyEge(kod[0], 'Обществознание')])

In [106]:
#Эвклидово расстояние
def EuclideanDistance(list1, list2):
    
    #Если списки не равны, то уравниваем их:
    if len(list1)!=len(list2):
        if len(list1)>len(list2):
            list2+=[0]*abs(len(list1)-len(list2))
        else: 
            list1+=[0]*abs(len(list1)-len(list2))
                           
    Eukl=0
    for i in range(len(list1)):
        Eukl+=(list1[i]-list2[i])**2
    return(Eukl**(1/2))

In [202]:
#Строим список к(20)-ближайших соседей для каждого студента
import numpy as np
itog_list=[]

for j in range(len(kod)):
    list_of_neibh=[]
    for i in range(len(ege_spis)):
        list_of_neibh.append([ege_spis[i][0],EuclideanDistance(ege_spis[j][1:], ege_spis[i][1:])])
    list_of_neibh.sort(key=lambda x: x[1])
    itog_list.append([ege_spis[j][0],list(np.array(list_of_neibh[1:21])[:,0])])

In [115]:
kod[:3]

[50095, 50104, 50137]

In [201]:
ege_spis

[[50095, 76, 56, 70],
 [50104, 72, 60, 70],
 [50137, 84, 68, 70],
 [50139, 84, 74, 70],
 [50152, 95, 48, 70],
 [50172, 71, 44, 70],
 [50193, 73, 72, 70],
 [50205, 79, 66, 70],
 [50209, 92, 79, 70],
 [50220, 87, 90, 70],
 [50222, 87, 81, 70],
 [50226, 65, 56, 70],
 [50230, 95, 68, 70],
 [50232, 71, 68, 70],
 [50239, 73, 40, 70],
 [50247, 100, 79, 70],
 [50248, 71, 44, 70],
 [50250, 71, 60, 70],
 [50280, 67, 79, 70],
 [50291, 79, 63, 70],
 [50308, 92, 87, 70],
 [50315, 55, 66, 70],
 [50336, 69, 72, 70],
 [50343, 71, 44, 70],
 [50350, 72, 68, 70],
 [50379, 70, 60, 70],
 [50481, 95, 74, 70],
 [50509, 71, 72, 70],
 [50513, 69, 70, 70],
 [50553, 68, 52, 70],
 [50557, 98, 77, 70],
 [50559, 68, 60, 70],
 [50563, 76, 60, 70],
 [50576, 90, 56, 70],
 [50610, 100, 94, 70],
 [50619, 61, 40, 70],
 [50627, 87, 79, 70],
 [50636, 73, 36, 70],
 [50655, 71, 70, 70],
 [50659, 84, 70, 70],
 [50662, 63, 68, 70],
 [50676, 82, 77, 70],
 [50689, 62, 60, 70],
 [50711, 63, 79, 70],
 [50712, 71, 52, 70],
 [50720,

In [215]:
itog_list.sort()
itog_list

[[50095,
  [50728.0,
   51224.0,
   66142.0,
   508887.0,
   510723.0,
   463433.0,
   465666.0,
   489906.0,
   490670.0,
   493591.0,
   494653.0,
   496161.0,
   308178.0,
   313474.0,
   313550.0,
   466430.0,
   472706.0,
   484015.0,
   490216.0,
   492291.0]],
 [50104,
  [52436.0,
   62738.0,
   63713.0,
   300599.0,
   320162.0,
   50250.0,
   56088.0,
   57585.0,
   59890.0,
   63671.0,
   300819.0,
   302447.0,
   302672.0,
   305525.0,
   308016.0,
   309493.0,
   315340.0,
   464903.0,
   469702.0,
   473278.0]],
 [50137,
  [60609.0,
   307257.0,
   465200.0,
   469324.0,
   472005.0,
   494056.0,
   504626.0,
   489763.0,
   490004.0,
   491851.0,
   508274.0,
   512751.0,
   513460.0,
   50659.0,
   51262.0,
   51419.0,
   51789.0,
   52691.0,
   56097.0,
   56348.0]],
 [50139,
  [50940.0,
   51731.0,
   460605.0,
   301013.0,
   301813.0,
   302077.0,
   303828.0,
   307496.0,
   489190.0,
   491143.0,
   509302.0,
   513257.0,
   303865.0,
   307378.0,
   308503.0,
   3

Теперь сделаем тоже самое для баллов за 1 семестр:

In [119]:


#Выводим оценки за выбранный предмет для каждого id:
#Ищем id студентов выбранной специальности
def markses(ID, predmet,semak):
    import sqlite3
 
    
    conn = sqlite3.connect("stud_spisok_ege.db")
    cursor = conn.cursor()
    
    #matan='Математический анализ'

    #выбираем прошедшие данные чтобы их вывести...
    sql = "SELECT ball FROM stud_spisok_ball WHERE stud_kod = ? AND predmet_name=? AND semestr=?"
    cursor.execute(sql,[(ID),(predmet),(semak)])
    #print(cursor.fetchall())
    marks=cursor.fetchall()
    
    
    sql_11 = "SELECT ball FROM stud_spisok_ball WHERE predmet_name=?"
    cursor.execute(sql_11,[(predmet)])
    maximum=cursor.fetchall()
    
    conn.commit()
    conn.close()
    
    new_11=0
    for i in maximum:
        if i[0]==None: continue
        if new_11<i[0]: new_11=i[0]

    new=[]
    for i in marks:
        if i[0]==None: new.append(0)
        else:new.append(i[0]*100/new_11)
    return(new) 

In [123]:
markses(kod[0],'Математический анализ',1)

[50.0, 87.5, 37.5]

In [199]:
marks_list=[]
for i in range(len(kod)):
    if(len(markses(kod[i],'Линейная алгебра',1))>0 and len(markses(kod[i],'Философия',1))>0 and len(markses(kod[i],'Математический анализ',1))>0 and len(markses(kod[129],'Психология',1))>0):
        marks_list.append([kod[i],markses(kod[i],'Философия',1)[-1],markses(kod[i],'Математический анализ',1)[-1],markses(kod[i],'Линейная алгебра',1)[-1],markses(kod[i],'Психология',1)[-1]])

In [200]:
marks_list

[[50095, 92.5, 37.5, 37.5, 80.0],
 [50104, 95.0, 37.5, 67.5, 60.0],
 [50137, 82.5, 37.5, 70.0, 80.0],
 [50139, 82.5, 42.5, 37.5, 80.0],
 [50152, 90.0, 42.5, 55.0, 60.0],
 [50172, 90.0, 37.5, 37.5, 66.0],
 [50193, 82.5, 70.0, 75.0, 80.0],
 [50205, 87.5, 60.0, 62.5, 80.0],
 [50209, 90.0, 62.5, 67.5, 76.0],
 [50220, 85.0, 47.5, 37.5, 80.0],
 [50222, 87.5, 55.0, 37.5, 80.0],
 [50226, 70.0, 37.5, 37.5, 60.0],
 [50230, 67.5, 37.5, 65.0, 80.0],
 [50232, 97.5, 40.0, 40.0, 44.0],
 [50239, 92.5, 70.0, 37.5, 60.0],
 [50247, 82.5, 37.5, 45.0, 60.0],
 [50248, 100.0, 62.5, 62.5, 80.0],
 [50250, 87.5, 55.0, 37.5, 80.0],
 [50280, 95.0, 62.5, 67.5, 80.0],
 [50291, 75.0, 37.5, 37.5, 40.0],
 [50308, 85.0, 50.0, 57.5, 80.0],
 [50315, 62.5, 37.5, 40.0, 40.0],
 [50336, 90.0, 37.5, 57.5, 80.0],
 [50343, 0, 27.5, 25.0, 0.0],
 [50350, 75.0, 37.5, 52.5, 40.0],
 [50379, 75.0, 37.5, 37.5, 76.0],
 [50481, 90.0, 75.0, 75.0, 80.0],
 [50509, 75.0, 37.5, 75.0, 60.0],
 [50513, 62.5, 37.5, 50.0, 60.0],
 [50553, 87.5, 52

In [206]:
#Строим список к(20)-ближайших соседей для каждого студента но уже по экзаменам
import numpy as np
itog_list_b=[]

for j in range(len(marks_list)):
    list_of_neibh_b=[]
    for i in range(len(marks_list)):
        list_of_neibh_b.append([marks_list[i][0],EuclideanDistance(marks_list[j][1:], marks_list[i][1:])])
    list_of_neibh_b.sort(key=lambda x: x[1])
    itog_list_b.append([marks_list[j][0],list(np.array(list_of_neibh_b[1:21])[:,0])])

In [217]:
itog_list_b.sort()
itog_list_b

[[50095,
  [54477.0,
   59900.0,
   62193.0,
   64626.0,
   51224.0,
   63605.0,
   50862.0,
   51350.0,
   51213.0,
   50139.0,
   54104.0,
   52429.0,
   64362.0,
   54816.0,
   54870.0,
   59136.0,
   50220.0,
   51896.0,
   55318.0,
   52378.0]],
 [50104,
  [55493.0,
   61840.0,
   59745.0,
   55896.0,
   62958.0,
   55162.0,
   66330.0,
   51705.0,
   62306.0,
   55735.0,
   51050.0,
   59345.0,
   52151.0,
   50676.0,
   58475.0,
   64368.0,
   57289.0,
   52691.0,
   53547.0,
   56729.0]],
 [50137,
  [50557.0,
   51092.0,
   50744.0,
   60985.0,
   56097.0,
   65364.0,
   62496.0,
   54000.0,
   54763.0,
   51073.0,
   53237.0,
   66142.0,
   53258.0,
   63417.0,
   67087.0,
   54007.0,
   51299.0,
   51731.0,
   52106.0,
   65421.0]],
 [50139,
  [51896.0,
   50862.0,
   50220.0,
   59900.0,
   64626.0,
   54816.0,
   50712.0,
   52369.0,
   56757.0,
   53263.0,
   52816.0,
   51213.0,
   53098.0,
   50379.0,
   54104.0,
   54463.0,
   51224.0,
   50095.0,
   53752.0,
   65389.0

In [208]:
len(itog_list)

3164

In [209]:
len(itog_list_b)

2132

In [228]:
mass=[]
for i in range(len(itog_list_b)):
    sch=0
    for j in range(len(itog_list)):
        if itog_list_b[i][0]==itog_list[j][0]:
            for k in range(len(itog_list[j])):
                if itog_list[j][k] in itog_list_b[i]:
                    sch+=1
            break
    mass.append(sch)

In [229]:
mass

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [220]:
s=0
for i in range(len(itog_list_b)):
    sch=0
    for j in range(len(itog_list_b[i])):
        if itog_list_b[i][0]==itog_list[j][0]:
            sch+=1
            break

In [221]:
s

129